In [144]:
import uproot
import numpy as np
import os
import matplotlib.pyplot as plt

"""liczba = os.popen('cd new_traning/; find -type f | wc -l').read()
liczba2 = int(liczba)"""

"liczba = os.popen('cd new_traning/; find -type f | wc -l').read()\nliczba2 = int(liczba)"

In [145]:
import glob
"""file_pattern = "new_traning/*.root"
files = glob.glob(file_pattern)"""

file = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/signal.root")
file2 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/VV_bkg0.root")
file3 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/VV_bkg1.root")
file4 = uproot.open("/home/szymon/ML_ex/autoencoding_pp/new_traning/VV_bkg2.root")

branch1 = file['nominal'].arrays(library="np")
branch2 = file2['nominal'].arrays(library="np")
branch3 = file3['nominal'].arrays(library="np")
branch4 = file4['nominal'].arrays(library="np")

In [146]:
#branch_list = [uproot.open(files[i])['nominal'].arrays(library="np") for i in range (liczba2) ]

branch_list = [branch1, branch2, branch3, branch4]

In [147]:
combined_data = {}

In [148]:
def delete_branches(branches):
    del branches['jet_isbtagged_DL1r_77']
    del branches['mc_kFactor']
    del branches['RunYear']
    del branches['mc_rawXSection']
    del branches['lep_ID_0']
    del branches['p_Event_No']
    del branches['jet_pt']
    del branches['jet_eta']
    del branches['jet_phi']
    del branches['jet_e']
    del branches['lep_vz_0']
    del branches['lep_vz_1']
    del branches['taus_DL1r_0']
    del branches['p_sumElepAll_EJet']
    del branches['p_sumPtleptauAll_Ptjet']
    del branches['best_Z_Mll']
    del branches['best_Z_other_MtLepMet']
    del branches['best_Z_other_Mll']
    del branches['minOSSFMll']
    del branches['minOSMll']
    del branches['totalEventsWeighted']
    del branches['weight_lumi_xsec']
    del branches['tau_charge_0']
    del branches['p_LBoostL1Tau_angle_farestJetToL2']
    del branches['eventNumber']
    del branches['p_is2lepTight']
    del branches['p_isTauSelected']
    del branches['tau_passEleBDT_0']
    del branches['tau_passMuonOLR_0']
    del branches['lep_ambiguityType_1']
    del branches['lep_ambiguityType_0']
    del branches['lep_plvWP_Tight_1']
    del branches['lep_plvWP_Tight_0']
    del branches['lep_isMedium_1']
    del branches['lep_isMedium_0']
    del branches['lep_isTightLH_1']
    del branches['lep_isTightLH_0']
    del branches['lep_d0_0'] # niekoniecznie usuniecie
    del branches['lep_BestMatchTrackEta_0']
    del branches['lep_BestMatchTrackPhi_0']
    del branches['lep_BestMatchTrackZ0_0']
    del branches['lep_BestMatchTrackQ_0']
    del branches['lep_chargeIDBDTLoose_0']
    del branches['lep_ID_1']
    del branches['lep_d0_1']
    del branches['lep_BestMatchTrackEta_1']
    del branches['lep_BestMatchTrackPhi_1']
    del branches['lep_BestMatchTrackZ0_1']
    del branches['lep_BestMatchTrackQ_1']
    del branches['lep_chargeIDBDTLoose_1']
    del branches['tau_numTrack_0']
    del branches['tau_passJetIDRNNMed_0']
    del branches['nJets_OR_DL1r_77']
    del branches['sublead_jetEta']
    del branches['sublead_jetPt']
    del branches['p_angle_LeadJetSubLeadJet']
    del branches['p_LBoost2L_tauPt']
    del branches['p_LBoost2L_DiffLep1Lep2phi']
    del branches['p_LBoost2L_AngleL1L2']
    del branches['p_LBoostL1Tau_DiffTauLep1phi']
    del branches['p_LBoostL1Tau_AngleL1Tau']
    del branches['p_LBoostL2Tau_DiffTauLep2phi']
    del branches['p_LBoostL2Tau_AngleL2Tau']

In [149]:
# p_LBoost2L_tauPt 
# p_LBoost2L_DiffLep1Lep2phi
# p_LBoost2L_AngleL1L2
# p_LBoostL1Tau_DiffTauLep1phi
# p_LBoostL2Tau_DiffTauLep2phi
# 

In [150]:
for i in branch_list:
    delete_branches(i)


In [151]:
keys = list(branch_list[0])

In [152]:
def join_branchKey(arr):
    combo = arr
    if len(arr) > 1:
        combo = np.concatenate([arr[0], arr[1]])
        for i in range(len(arr)):
            combo = np.concatenate([combo, arr[i]])
    return combo

In [153]:
def join_branch(branch):
    try:
        for key in keys:
            if all(isinstance(item, np.ndarray) for item in branch[key]):
                branch[key] = join_branchKey(branch[key])
        return branch
    except IndexError:
        return branch

In [ ]:
for i in range(len(branch_list)):
    branch_list[i] = join_branch(branch_list[i])

In [156]:
combined_data

{}

In [157]:
for i in keys:
    combo = np.concatenate([branch_list[0][i], np.array([])])
    for k in branch_list:
        combo = np.concatenate([combo, k[i]])
    combined_data[i] = combo
    try:
        combined_data[i] = combined_data[i].astype(np.float32)
    except ValueError:
        print(i)

In [201]:
def data_before():
    ex = []
    std = []
    min = []
    max = []
    std5 = []
    
    for key in keys:
        ex.append(np.mean(combined_data[key]))
        std.append(np.std(combined_data[key]))
        min.append(np.min(combined_data[key]))
        max.append(np.max(combined_data[key]))
        std5.append(5*np.std(combined_data[key]))
        
    return ex, std, min, max, std5

In [158]:
for i in range(len(keys)):
    if np.min(combined_data[keys[i]]) == -999.0:
        del combined_data[keys[i]]

In [159]:
for key in keys:
    if "_inv" in key:
        for i in range(len(combined_data[key])):
            if combined_data[key][i] < 0:
                combined_data[key][i] = 0

In [160]:
keys = list(combined_data)

In [205]:
ex, std, min, max, std5 = data_before()

In [ ]:
for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")

In [161]:
def paint(m, lst, color, key):
    fig = plt.figure()
    biny = bins(m, lst)
    plt.hist(lst, bins=biny, color=color)
    plt.title(key)
    return fig

In [162]:
def bins(n, list):
    bins = []

    list = sorted(list)

    last = list[-1]
    k = np.min(list)
    while k < last:
        bins.append(k)
        k += (last - list[0])/n


    return bins

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('plots_before2.pdf')
for i in keys[102:]:
    fig = paint(100, combined_data[i], "red", i)
    plt.savefig(pp, format='pdf')
pp.close()    

In [163]:
def cuting(list):
    mean = np.mean(list)
    std = np.std(list)
    
    filtred = []
    
    max_list = []
    min_list = []
    
    max_plus = mean + 5*std
    max_minus = mean - 5*std
    for i in range(len(list)):
        if(list[i] <= max_plus and list[i] >= max_minus):
            filtred.append(list[i])
        elif (list[i] >= max_plus):
          max_list.append(list[i])
        elif (list[i] <= max_minus):
            min_list.append(list[i])
    if (len(max_list) > 0):
        cut_max = np.min(max_list)
    else:
        cut_max = np.max(filtred)
    if(len(min_list) > 0):
        cut_min = np.max(min_list)
    else:
        cut_min = np.min(filtred)
    
    for i in range(len(max_list)):
        filtred.append(max_plus)
    
    for i in range(len(min_list)):
        filtred.append(max_minus)
    
    return [np.asarray(filtred), cut_max, cut_min, max_list, min_list]

In [ ]:
for key in keys:
    cut = cuting(combined_data[key])
    print(f'{key} -- filtered: {len(cut[0])} right cut num: {len(cut[3])}, left cut num: {len(cut[4])}')

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('plots_before3.pdf')
for i in keys[202:]:
    fig = paint(i, 100, combined_data, "red")
    plt.savefig(pp, format='pdf')
pp.close()

In [164]:
def minus_value_filter(keys, branch):
    try:
        for key in keys:
            min = np.min(branch[key])
            max = np.max(branch[key])
            distance = abs(max - min)
            if min < 0:
                for i in range(len(branch[key])):
                    branch[key][i] = branch[key][i] + distance
        return branch
    except ValueError:
        print(key)

In [165]:
combined_data = minus_value_filter(keys, combined_data)


In [166]:
def norm(keys, branch):
    
    maxes = {}
    mins = {}
    
    for key in keys:
        max = np.max(branch[key])
        min = np.min(branch[key])
        distance = max - min
        
        for i in range(len(branch[key])):
            if distance != 0:
                branch[key][i] = (branch[key][i] - min) / distance
        if(max != 0):
            maxes[key] = (max - min) / distance   
            mins[key] = 0
        
        
    return [branch, maxes, mins]

In [26]:
from sklearn import preprocessing

def new_norm(keys, branch):
    
    for key in keys:
        branch[key] = preprocessing.normalize(branch[key].reshape(1, -1))
        branch[key] = branch[key].T
        
    return branch

In [167]:
combined_data = norm(keys, combined_data)[0]

In [168]:
globals_max = {}
globals_min = {}

for key in keys:
    
    cut = cuting(combined_data[key])
    combined_data[key] = cut[0]
    globals_max[key] = cut[1]
    globals_min[key] = cut[2]


In [ ]:
ex, std, min, max, std5 = data_before()

for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")

In [124]:
def paint2(key, m, lst, lst2, color, color2, alpha):
    fig = plt.figure()
    biny = bins(m, lst[key])
    plt.hist(lst[key], bins=biny, color=color)
    plt.hist(lst2[key], bins=biny, color=color2, alpha=alpha)
    plt.title(key)
    return fig

In [ ]:
pp = PdfPages('plots_after1.pdf')
for i in keys[:101]:
    fig = paint2(i, 100, background, "blue")
    plt.savefig(pp, format='pdf')
pp.close()

In [ ]:
list_keys = []
filtered_cor = []

sum = 0
for key in keys:
    a = combined_data[key]
    list_keys.append(key)
    for k in keys:
        try:
            if k != key and k not in list_keys:
                b = combined_data[k]
                cor_coff = np.corrcoef(a.reshape(1, -1), b.reshape(1, -1))
                if abs(cor_coff[0, 1]) > 0.8:
                    print(f'{key} with {k}: {cor_coff[0, 1]}')
                    sum += 1
                    if key not in filtered_cor:
                        filtered_cor.append(key)
        except AttributeError:
            print(f'{key}: {a}')
            print(f'{k}: {b}')
        
                
print(sum)
print(filtered_cor)
print(len(filtered_cor))

In [170]:
fil_keys = []

for key in keys:
    if key not in filtered_cor:
        fil_keys.append(key)
        
print(len(fil_keys))

196


In [171]:
keys = fil_keys

In [172]:
signal = branch_list[0]
background = {}
for key in keys:
    background[key] = combined_data[key][len(signal[key]):]
    signal[key] = combined_data[key][:len(signal[key])]


In [173]:
signal_filtered = {}
background_filtered = {}
combined_data_filtered = {}

In [174]:
for key in keys:
    signal_filtered[key] = signal[key]
    background_filtered[key] = background[key]  
    combined_data_filtered[key] = combined_data[key]

In [178]:
np.save("new_signal_data.npy", signal_filtered)
np.save("new_background_data.npy", background_filtered)
np.save("new_combined_data.npy", combined_data_filtered)

In [ ]:
pp = PdfPages('plots_after2.pdf')
for i in keys[101:]:
    fig = paint2(i, 100, background_filtered, signal_filtered, "blue", "red", 0.6)
    plt.savefig(pp, format='pdf')
pp.close()

In [ ]:
for i in range(10):
    paint(100, signal_filtered[i], )